## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [68]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def BandpassFilter(signal, pass_band, fs=125):
    """Bandpass fiter
    Args:
        signal: (array) the input signal
        pass_band: (tuple) pass band
        fs: (number) the sampling rate of signal
    Returns:
        (array) the filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def PlotSpec(signal, fs = 125):
    """
    Plot the spectogram for a given signal
    """
    plt.figure(figsize(12, 8))
    plt.specgram(signal, Fs=fs, NFFT=250, noverlap=125, xextent=[0, len(signal) / fs / 60])
    plt.xlabel("Time (min)")
    plt.ylabel("Frequency (Hz)")
    
def CalcAccMag(accx, accy, accz):
    """
    Calculate the accelerometer magnitude
    """
    return np.sqrt(np.sum([np.square(accx), np.square(accy), np.square(accz)], axis=0))


def PulseRateInter(ppg, acc, tolerance =2, fs = 125):
    '''
    Calculate pulse rate and confidence at each interval
    '''

    # bandpass all signals
    low = 40 / 60
    high = 240 / 60
    pass_band = (low, high)
    ppg_f = BandpassFilter(ppg, pass_band)
    acc_f = BandpassFilter(acc, pass_band)
    
    # calculate the fourier transform for the accelerometer
    acc_len = len(acc_f)
    acc_freq = np.fft.rfftfreq(acc_len, 1/fs)
    acc_fft_mag = np.abs(np.fft.rfft(acc_f, acc_len))
    
    # calculate the fourier transform for the ppg
    ppg_len = len(ppg_f)
    ppg_freq = np.fft.rfftfreq(ppg_len, 1/fs)
    ppg_fft_mag = np.abs(np.fft.rfft(ppg_f, ppg_len))
    
    # find dominant accelerometer frequency if same as ppg, use the next ppg
    # if not available then same as accelerometer
    acc_frep_max = acc_freq[np.argmax(acc_fft_mag)]
    ppg_frep_max = ppg_freq[np.argmax(ppg_fft_mag)]
        
    if (acc_frep_max == ppg_frep_max) and len(ppg_fft_mag) >=2:
        pulse_freq = ppg_freq[np.argsort(ppg_fft_mag)[::-1][1]]
        pulse_rate = pulse_freq * 60
        
    else:
        pulse_freq = ppg_frep_max
        pulse_rate = pulse_freq * 60
    
    
    # confidence: how much energy is concentrated near the pulse rate?
    ## summing the frequency spectrum near the pulse rate estimate and dividing it by the sum of the entire spectrum
    spectral_energy_ppg = np.square(ppg_fft_mag)
    interval = 5/60 # more or less 5 BPM
    confidence = (np.sum(spectral_energy_ppg[(ppg_freq >= (pulse_freq - interval)) & (ppg_freq <= (pulse_freq + interval))])) / np.sum(spectral_energy_ppg)
    
    return pulse_rate, confidence
    

def RunPulseRateAlgorithm(data_fl, ref_fl, fs = 125):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Load ref
    ref = sp.io.loadmat(ref_fl)['BPM0']#.reshape(-1)
    
    
    # get the accelerometer magnitude
    acc = CalcAccMag(accx, accy, accz)
    
    # get the intervals
    window_length_s = 6
    window_shift_s = 2
    window_length = window_length_s * fs
    window_shift = window_shift_s * fs
    
    errors, confidences = [], []
    
    # truncate to get right dimension
    rem = (len(ppg) - window_length) % window_shift
    ppg = ppg[:-rem]
    acc = acc[:-rem]
    
    for j, i in enumerate(range(0, len(ppg) - window_length - rem, window_shift)):
        ppg_w = ppg[i: i + window_length]
        acc_w = acc[i: i + window_length]
        
        # get the pulse rate estimates and confidence
        pulse_rate, confidence = PulseRateInter(ppg_w, acc_w, fs = fs)
        error = np.abs(ref[j][0] - pulse_rate)
        
        # append to each list
        errors.append(error)
        confidences.append(confidence)
    

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    #errors, confidence = np.ones(100), np.ones(100)  # Dummy placeholders. Remove
    return errors, confidences

In [69]:
Evaluate()

28.25374046974623

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code Description**
The code above allows to isolate the heart rate of people from Wrist-Type signals during intensive physical exercise. Just need to run the Evaluate function to get performance of the algorithm

**Data Description**
The data comprises 2 main sources: PPG signal and a 3-axis accelerometer

**Algorithhm Description**
The algorithm takes the data from both sources, identify the main frequences from the PPG and remove the frequences that comes from movements in order to isolate the heart beat only
It takes advantage of the PPG sensor of the light emitted by the PPG sensor that is absorbed by red blood cells in these capillaries. The period of this oscillating waveform is the pulse rate
We used heurisitcs to remove frequencies that don't pertain to heart beat such as movements identified from the accelerometer. This is a simplification from the reality furthermore that the frequencies can be the same.

**Algorithm Performance**
We used a test set to calculate the performance of the algorithm. It's unclear if the test set includes people from different skin colors or other conditions so the generalizability of the performance is to take with a grain of salt.



-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 